# Next‑Level NumPy (Post‑Basics)

This notebook builds on fundamentals and focuses on the mental models that make NumPy fast, correct, and expressive.


In [4]:
import numpy as np

np.set_printoptions(precision=4, suppress=True)


## 1. Shape Reasoning & Broadcasting (Top Priority)
Broadcasting rules (right‑alignment, size‑1 expansion) and explicit dimension insertion.


In [5]:
# Right‑alignment and size‑1 expansion
A = np.arange(6).reshape(2, 3)   # shape (2,3)
print(A, end="\n\n")
b = np.array([10, 20, 30])       # shape (3,)
print(b)

A + b   # b is treated as (1,3) then expanded to (2,3)

[[0 1 2]
 [3 4 5]]

[10 20 30]


array([[10, 21, 32],
       [13, 24, 35]])

In [6]:
# Explicit dimension insertion
x = np.array([1, 2, 3])          # (3,)

print(x[:, None].shape)                 # (3,1)
print(x[None, :].shape)                 # (1,3)


(3, 1)
(1, 3)


In [7]:
# Predict output shape mentally
X = np.random.randn(5, 3)        # (n=5, d=3)

print((X - X.mean(axis = 0)) / X.std(axis = 0))     # convert to z-scores


[[ 0.7155  0.2033 -0.2086]
 [-0.9517  0.9211  0.9539]
 [ 1.4667 -0.6457 -1.6876]
 [-1.1958  1.0983  1.0853]
 [-0.0347 -1.577  -0.1431]]


In [8]:
# Pairwise ops without loops
X = np.array([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])  # (3,2)

pairwise_diff = X[:, None, :] - X[None, :, :]      # (3,3,2)
pairwise_diff

array([[[ 0.,  0.],
        [-2., -2.],
        [-4., -4.]],

       [[ 2.,  2.],
        [ 0.,  0.],
        [-2., -2.]],

       [[ 4.,  4.],
        [ 2.,  2.],
        [ 0.,  0.]]])

In [9]:
# exercise part 1
squared_diff = np.square(X[:, None, :] - X[None, :, :])
squared_diff

array([[[ 0.,  0.],
        [ 4.,  4.],
        [16., 16.]],

       [[ 4.,  4.],
        [ 0.,  0.],
        [ 4.,  4.]],

       [[16., 16.],
        [ 4.,  4.],
        [ 0.,  0.]]])

In [10]:
# exercise part 2
manhattan = np.abs(X[:, None, :] - X[None, :, :]).sum(axis=2)
manhattan

array([[0., 4., 8.],
       [4., 0., 4.],
       [8., 4., 0.]])

## 2. Axis Semantics (Beyond “axis=0 means columns”)
Reduction vs transformation, `keepdims=True`, and behavior under reshape/transpose.


In [11]:
X = np.arange(1, 13).reshape(4, 3)
X


array([[ 1,  2,  3],
       [ 4,  5,  6],
       [ 7,  8,  9],
       [10, 11, 12]])

In [12]:
# Reduction along axis
print(*X.sum(axis=0))    # per‑feature (column) sum
print(*X.sum(axis=1) )          # per‑sample (row) sum


22 26 30
6 15 24 33


In [13]:
# keepdims preserves dimensionality for broadcasting
mean0 = X.mean(axis=0, keepdims=True)  # (1,3)
X_centered = X - mean0                 # broadcasts correctly
X_centered


array([[-4.5, -4.5, -4.5],
       [-1.5, -1.5, -1.5],
       [ 1.5,  1.5,  1.5],
       [ 4.5,  4.5,  4.5]])

In [14]:
# Row‑wise normalization
row_norms = np.linalg.norm(X, axis=1, keepdims=True)
X_normalized = X / row_norms
X_normalized


array([[0.2673, 0.5345, 0.8018],
       [0.4558, 0.5698, 0.6838],
       [0.5026, 0.5744, 0.6462],
       [0.5234, 0.5758, 0.6281]])

In [15]:
# Axis behavior under transpose
Xt = X.T
X.shape, Xt.shape


((4, 3), (3, 4))

## 3. Views, Copies, and Fancy Indexing Traps
Slicing usually returns a **view**; fancy indexing always returns a **copy**.


In [16]:
arr = np.arange(10)
view = arr[2:7]       # slice view
copy = arr[[2, 3, 4]] # fancy index copy

np.shares_memory(arr, view), np.shares_memory(arr, copy)


(True, False)

In [17]:
view[:] = 99
arr


array([ 0,  1, 99, 99, 99, 99, 99,  7,  8,  9])

In [18]:
copy[:] = 77
arr  # unchanged by copy edits


array([ 0,  1, 99, 99, 99, 99, 99,  7,  8,  9])

In [19]:
# base attribute points to original owning array (or None)
print(view.base)
view.base is arr, copy.base


[ 0  1 99 99 99 99 99  7  8  9]


(True, None)

## 4. Memory Layout & Strides (Mental Model)
Contiguity and strides explain why some ops are free and others allocate.


In [20]:
X = np.arange(12).reshape(3, 4)
X.flags['C_CONTIGUOUS'], X.flags['F_CONTIGUOUS'], X.strides


(True, False, (32, 8))

In [21]:
Xt = X.T
Xt.flags['C_CONTIGUOUS'], Xt.flags['F_CONTIGUOUS'], Xt.strides


(False, True, (8, 32))

In [22]:
# Reshape can be free if memory is compatible
Y = X.reshape(2, 6)
Y


array([[ 0,  1,  2,  3,  4,  5],
       [ 6,  7,  8,  9, 10, 11]])

In [23]:
# Transpose often returns a view with different strides
Xt = X.T
Xt


array([[ 0,  4,  8],
       [ 1,  5,  9],
       [ 2,  6, 10],
       [ 3,  7, 11]])

(Optional) `as_strided` can create sliding windows without copying.
Use with care — incorrect strides can crash Python or return invalid data.


In [24]:
from numpy.lib.stride_tricks import as_strided

x = np.arange(8)
# Create a 5x4 sliding window view over x
window_shape = (5, 4)
strides = (x.strides[0], x.strides[0])
windows = as_strided(x, shape=window_shape, strides=strides)
windows


array([[0, 1, 2, 3],
       [1, 2, 3, 4],
       [2, 3, 4, 5],
       [3, 4, 5, 6],
       [4, 5, 6, 7]])

In [25]:
# exercise
x = np.arange(10)
window_shape = (7, 4)
strides = (x.strides[0], x.strides[0])
windows = as_strided(x, shape=window_shape, strides=strides)
windows

array([[0, 1, 2, 3],
       [1, 2, 3, 4],
       [2, 3, 4, 5],
       [3, 4, 5, 6],
       [4, 5, 6, 7],
       [5, 6, 7, 8],
       [6, 7, 8, 9]])

In [26]:
# exercise
X = np.arange(12).reshape(3, 4)
as_strided(X[:, ::-1], shape=(3, 4), strides=(X.strides[0], -X.strides[1]))

array([[ 3,  2,  1,  0],
       [ 7,  6,  5,  4],
       [11, 10,  9,  8]])

## 5. Vectorization Patterns (Algorithmic Thinking)
Broadcast + reduce, mask + assign, reshape → operate → reshape back.


In [27]:
# Pairwise distance matrix (squared Euclidean)
X = np.random.randn(4, 3)

# (n,1,d) - (1,n,d) -> (n,n,d), then sum over d
D2 = ((X[:, None, :] - X[None, :, :]) ** 2).sum(axis=2)
D2


array([[ 0.    ,  2.7215, 19.2888, 15.3924],
       [ 2.7215,  0.    , 10.775 , 10.0618],
       [19.2888, 10.775 ,  0.    ,  4.1938],
       [15.3924, 10.0618,  4.1938,  0.    ]])

In [28]:
# Mask + assign
x = np.linspace(-2, 2, 9)
y = x.copy()

y[x < 0] = 0

y


array([0. , 0. , 0. , 0. , 0. , 0.5, 1. , 1.5, 2. ])

In [29]:
# Reshape -> operate -> reshape back
img = np.arange(3*4*2).reshape(3, 4, 2)  # (H,W,C)
flat = img.reshape(-1, 2)               # (H*W, C)
flat = flat - flat.mean(axis=0, keepdims=True)
img_centered = flat.reshape(3, 4, 2)
img_centered


array([[[-11., -11.],
        [ -9.,  -9.],
        [ -7.,  -7.],
        [ -5.,  -5.]],

       [[ -3.,  -3.],
        [ -1.,  -1.],
        [  1.,   1.],
        [  3.,   3.]],

       [[  5.,   5.],
        [  7.,   7.],
        [  9.,   9.],
        [ 11.,  11.]]])

## 6. Boolean Logic as Control Flow
Compound masks, `np.where`, and data filtering pipelines.


In [30]:
x = np.linspace(-3, 3, 13)

mask = (x >= -1) & (x <= 1)
filtered = x[mask]

mask, filtered


(array([False, False, False, False,  True,  True,  True,  True,  True,
        False, False, False, False]),
 array([-1. , -0.5,  0. ,  0.5,  1. ]))

In [31]:
# Piecewise function using np.where
f = np.where(x < 0, x**2, np.sqrt(x))
f


/var/folders/t5/pnfh13sd6mx11zzqg0jys72w0000gn/T/ipykernel_19963/3442962931.py:2: RuntimeWarning: invalid value encountered in sqrt
  f = np.where(x < 0, x**2, np.sqrt(x))


array([9.    , 6.25  , 4.    , 2.25  , 1.    , 0.25  , 0.    , 0.7071,
       1.    , 1.2247, 1.4142, 1.5811, 1.7321])

In [32]:
# Clipping and thresholding
np.clip(x, -1, 1)


array([-1. , -1. , -1. , -1. , -1. , -0.5,  0. ,  0.5,  1. ,  1. ,  1. ,
        1. ,  1. ])

## 7. Numerical Linear Algebra (Used Correctly)
Avoid explicit inverse; prefer `solve`, `lstsq`, and `pinv`.


In [33]:
A = np.array([[3.0, 1.0], [1.0, 2.0]])
b = np.array([9.0, 8.0])

x = np.linalg.solve(A, b)
x


array([2., 3.])

In [34]:
# Least squares (overdetermined)
X = np.array([[1, 1], [1, 2], [1, 3]], dtype=float)
y = np.array([1, 2, 2], dtype=float)

beta, residuals, rank, s = np.linalg.lstsq(X, y, rcond=None)
beta, residuals, rank


(array([0.6667, 0.5   ]), array([0.1667]), np.int32(2))

In [35]:
# Rank deficiency and pseudoinverse
A = np.array([[1, 2], [2, 4]], dtype=float)  # rank 1
np.linalg.matrix_rank(A), np.linalg.pinv(A)


(np.int64(1),
 array([[0.04, 0.08],
        [0.08, 0.16]]))

In [36]:
# PCA without libraries via SVD
X = np.random.randn(100, 3)
X = X - X.mean(axis=0, keepdims=True)

U, S, Vt = np.linalg.svd(X, full_matrices=False)
components = Vt   # principal axes
explained_variance = (S**2) / (len(X) - 1)
components, explained_variance


(array([[ 0.0538, -0.276 , -0.9596],
        [ 0.5487, -0.7948,  0.2594],
        [ 0.8343,  0.5405, -0.1087]]),
 array([1.1675, 0.8896, 0.7857]))

## 8. Numerical Stability & Dtypes
Floating‑point error, overflow/underflow, stable formulations.


In [37]:
# float32 vs float64
x32 = np.array([1e10, 1.0], dtype=np.float32)
x64 = np.array([1e10, 1.0], dtype=np.float64)

x32 - 1e10, x64 - 1e10


(array([ 0.e+00, -1.e+10], dtype=float32), array([ 0.e+00, -1.e+10]))

In [38]:
# Stable softmax
z = np.array([1000.0, 1001.0, 1002.0])

exp_naive = np.exp(z) / np.exp(z).sum()
exp_stable = np.exp(z - z.max()) / np.exp(z - z.max()).sum()

exp_naive, exp_stable


/var/folders/t5/pnfh13sd6mx11zzqg0jys72w0000gn/T/ipykernel_19963/2633921692.py:4: RuntimeWarning: overflow encountered in exp
  exp_naive = np.exp(z) / np.exp(z).sum()
/var/folders/t5/pnfh13sd6mx11zzqg0jys72w0000gn/T/ipykernel_19963/2633921692.py:4: RuntimeWarning: invalid value encountered in divide
  exp_naive = np.exp(z) / np.exp(z).sum()


(array([nan, nan, nan]), array([0.09  , 0.2447, 0.6652]))

In [39]:
# Comparing floats safely
a = 0.1 + 0.2
b = 0.3
a == b, np.isclose(a, b)


(False, np.True_)

## 9. Randomness & Simulation (Modern NumPy)
Use `default_rng` for reproducibility and performance.


In [40]:
rng = np.random.default_rng(123)

rng.random(5)


array([0.6824, 0.0538, 0.2204, 0.1844, 0.1759])

In [41]:
# Monte Carlo estimate of pi
rng = np.random.default_rng(0)
N = 1000000
pts = rng.random((N, 2))
inside = (pts**2).sum(axis=1) <= 1.0
pi_est = 4 * inside.mean()
pi_est


np.float64(3.141168)

In [56]:
# Random walk (vectorized)
steps = rng.choice([-1, 1], size=10000)
walk = steps.cumsum()
walk[-1]


np.int64(122)

## 10. `einsum` (Optional but Elite)
Express complex operations compactly.


In [57]:
A = np.array([[1, 2], [3, 4]])
B = np.array([[5, 6], [7, 8]])

np.einsum('ij,jk->ik', A, B)  # matrix multiplication


array([[19, 22],
       [43, 50]])

In [58]:
# Trace and Frobenius norm
np.einsum('ii', A)            # trace
np.sqrt(np.einsum('ij,ij', A, A))


np.float64(5.477225575051661)

## 11. Performance Awareness
Avoid unnecessary temporaries, check contiguity, and time critical paths.


In [59]:
import timeit

X = np.random.randn(1000, 1000)

# Two ways to compute row-wise mean centering

def center_naive():
    return X - X.mean(axis=0)

def center_keepdims():
    return X - X.mean(axis=0, keepdims=True)

# keepdims avoids reshaping or broadcasting mistakes, same speed here
naive_time = timeit.timeit(center_naive, number=5)
keepdims_time = timeit.timeit(center_keepdims, number=5)

naive_time, keepdims_time


(0.0033704170491546392, 0.0036623330088332295)

In [60]:
# Contiguity checks
X.flags['C_CONTIGUOUS'], X.T.flags['C_CONTIGUOUS']


(True, False)